# Decison Support System
### Deciding Factors
- Longevity: time since review
- Length of title and text: count the number of words
- Helpfulness of a review: ratio of “helpful votes” to “total votes”
- Readership: total number of votes of a review
- Select products wtih atleast 100 reviews, eliminated the reviews that had less than 4 votes
- Polarity = positive sentiment + negative sentiment
- Sentiment = positive sentiment - negative sentiment - 2:

In [ ]:
import pandas as pd
import numpy as np
import gzip 
import json
import shutil
import time

In [ ]:
def getSentimentsAndLengthFromFile(nameOfFile):
    df=pd.DataFrame()
    file = open(nameOfFile, encoding = "utf8")
    count = 0
    column_names = []
    for line in file:
        count=count+1
        if count is 1:
            column_names = line.split('\t')
            df=pd.DataFrame(columns = column_names)
        temp_line = line.rstrip().split('\t')
        df = df.append(pd.Series(temp_line ,index = column_names), ignore_index = True)
    file.close()
    return df

def unzipJson(filename, saveas):
    "both arguments are strings"
    with gzip.open(filename, 'rb') as f_in:
        with open(saveas, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
            
def getDataFromJsonFile(filename, noOfRev):
    count = 0
    df = pd.DataFrame()
    with open(filename, 'r') as f:
        for line in f:
            if count == noOfRev:
                break
            count = count + 1
            if count%500==0:
                print(count)
            data = json.loads(line)
            dictionary=json.loads(line)
            dictionary["helpful"] = str(dictionary["helpful"][0])+":"+str(dictionary["helpful"][1])
            df = df.append(pd.DataFrame(dictionary,index = [count]), sort = False)
    return df

#### Description of the below cell
- unzip the json.gz file 
- get a dataFrame consisting of 35000 reviews
- store this dataFrame in a csv file
- now this csv file is converted to a txt file that is then
    analysed using the sentistrength software, resulting is a
    txt file having analysed result

In [ ]:
# unzipJson("reviews_Electronics_5.json.gz","reviews_Electronics_5.json")
# df = getDataFromJsonFile("reviews_Electronics_5.json",35000)
# df.to_csv("dataset_electronic.csv")


#### Description of the below cell
- dataset_analysed.txt contains all the analysed data that is converted to a dataFrame
- dataFrame is stored in a csv file for faster access next time


In [ ]:
# data = getSentimentsAndLengthFromFile("dataset_analysed.txt")

In [ ]:
# data = data.drop(0,0)
# data.to_csv("dataset.csv")

In [ ]:
# data['polarityText'] = data['reviewTextPositive'] + data['reviewTextNegative']
# data['polaritySummary'] = data['summaryPositive'] + data['summaryNegative\n']
# data['sentimentText'] = data['reviewTextPositive'] - data['reviewTextNegative']-2
# data['sentimentSummary'] = data['summaryPositive'] - data['summaryNegative\n'] - 2
# data['summaryLength'] = data['summary'].str.split().str.len()
# data['reviewTextLength'] = data['reviewText'].str.split().str.len()
# data['readership'] = data['helpful'].str.split(':').str[1]
# data['longetivity'] = time.time() - data['unixReviewTime']
# del data['no']

In [ ]:
# data.to_csv("dataset.csv")

In [66]:
# read the dataset
data = pd.read_csv('dataset.csv')

In [67]:
# Select products wtih atleast 100 reviews, eliminated the reviews that had less than 4 votes
tempDf = data.groupby('asin').size().rename("countOfReviews").reset_index()
data = data.merge(tempDf)
data = data[data['countOfReviews'] >= 80]
data = data.loc[data['readership'] >= 3]
data

,no,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,...,summaryNegative,polarityText,polaritySummary,sentimentText,sentimentSummary,summaryLength,reviewTextLength,readership,longetivity,countOfReviews
22,22,AVRFGGCCCR6QU,0972683275,"""Alberto Dieguez """"premiere purchaser of rando...",3:4,This wall mount does everything it's supposed ...,4.0,Fairly good wall mount,1283126400,"08 30, 2010",...,-1,0,1,0,1,4,81.0,4,2.686830e+08,219
26,26,A26QCCO0101CO1,0972683275,"""Amazon Customer """"jkhinch""""""",7:9,I used this for my 47&#34; Samsung. Its fit/fe...,5.0,Just what I'd hoped,1361404800,"02 21, 2013",...,-1,1,1,3,1,4,79.0,9,1.904046e+08,219
56,56,A3QH8VQDE7HZCR,0972683275,costaricachris,15:19,Quality was excellent. Instructions were clear...,5.0,Real value for the money,1286236800,"10 5, 2010",...,-1,3,1,3,1,5,64.0,19,2.655726e+08,219
82,82,A38FGQVJM18OWV,0972683275,"""George S. Mitchell """"gsmitchell""""""",8:18,I checked around Amazon as well as some other ...,1.0,What a piece of junk!,1291161600,"12 1, 2010",...,-2,0,-1,2,1,5,138.0,18,2.606478e+08,219
141,141,A7R4R9THNELTP,0972683275,LG,14:19,"""I mounted this in my RV and it holds a 26"""" L...",5.0,"Excellent, at any price",1285804800,"09 30, 2010",...,-1,0,3,4,3,4,47.0,19,2.660046e+08,219
145,145,A3SXYB8JPQ44LN,0972683275,Marcel,3:4,"""Used this for a Panasonic 32"""" LCD TV. Mount...",4.0,"Good product, poor instructions",1342483200,"07 17, 2012",...,-2,1,0,3,2,4,95.0,4,2.093262e+08,219
155,155,A3AKVALGT4Y02G,0972683275,MICHAEL,5:6,"For the money you can't go wrong. Yes, it has ...",4.0,can't beat it for the price,1203379200,"02 19, 2008",...,-1,2,0,2,0,6,44.0,6,3.484302e+08,219
156,156,A2I8J6GW1H6LWU,0972683275,Michael,7:9,I love this mount! All the adjustments for til...,5.0,Great low cost mount!,1285459200,"09 26, 2010",...,-2,2,1,2,3,4,28.0,9,2.663502e+08,219
167,167,A15K7HV1XD6YWR,0972683275,Musawir Karim,5:7,"""I consider myself to be a good DIYer. I've in...",1.0,Beware the TV will not center....folding arm l...,1360713600,"02 13, 2013",...,-2,-1,-1,5,1,10,214.0,7,1.910958e+08,219
201,201,A11ZP93TJED4T6,0972683275,Ryan,1:8,The fact that the pivot point on this TV mount...,2.0,poor manufacture quality..,1285891200,"10 1, 2010",...,-2,-1,-1,1,1,3,77.0,8,2.659182e+08,219
